# Start

In [17]:
import os

os.chdir("/net/tscratch/people/plglukaszst/projects/ensemble-ai2024/defensetransformation")

In [18]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
from torch import nn
from torchmetrics.classification import Accuracy
from datetime import datetime
import requests

# Data

In [19]:
data_evaluate = np.load(
    "data/DefenseTransformationEvaluate.npz"
)

data_submit = np.load("data/DefenseTransformationSubmit.npz")

In [20]:
LABELS_TRAIN = data_evaluate['labels']
REPR_TRAIN = data_evaluate['representations']
REPR_TEST = data_submit['representations']

In [21]:
class DefenseTransformationDataset(Dataset):
    def __init__(self, repr: np.ndarray, labels: np.ndarray):
        self.repr = repr
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        repr = torch.from_numpy(self.repr[idx])
        labels = torch.tensor(self.labels[idx])
        return repr, labels

In [22]:
def get_dl(ds: Dataset, batch_size: int) -> DataLoader:
    gen = torch.Generator()
    gen.manual_seed(0)
    return DataLoader(ds, batch_size=batch_size, shuffle=True, generator=gen)

In [29]:
N_EPOCHS = 20

def finetune_classifier(model, dataloader):
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    loss_fn = nn.CrossEntropyLoss()
    acc = Accuracy(num_classes=10, task='multiclass')

    for epoch in range(N_EPOCHS):
        last_loss = 0.0
        model.train()
        for batch_idx, batch in enumerate(dataloader):
            repr, labels = batch
            repr, labels = repr.to("cuda"), labels.to("cuda")

            optimizer.zero_grad()
            outputs = model(repr)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            acc.reset()
            for batch_idx, batch in enumerate(dataloader):
                repr, labels = batch
                repr, labels = repr.to("cuda"), labels.to("cuda")

                outputs = model(repr)
                loss = loss_fn(outputs, labels)
                pred = outputs.argmax(dim=1).cpu()
                acc.update(pred, labels.cpu())

            print('Epoch {} loss: {}, accuracy: {}'.format(epoch, last_loss, acc.compute()))


In [30]:
class Task3Model(nn.Module):
    def __init__(self, feat_size=192):
        super().__init__()
        self.feat_size=feat_size
        self.layer1 = nn.Sequential(nn.Linear(in_features=self.feat_size, out_features=128, bias=True), nn.Tanh())
        self.layer2 = nn.Sequential(nn.Linear(in_features=128, out_features=32, bias=True), nn.Tanh())
        self.layer3 = nn.Sequential(nn.Linear(in_features=32, out_features=10, bias=True))

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

In [193]:
def transform_representations(repr: np.ndarray) -> np.ndarray:
    def create_affine_transformation_matrix(size):
        np.random.seed(42)  # Ensure reproducibility
        # Creating a random matrix for linear transformation part
        A = np.eye(size) + np.random.normal(0, 0.1, (size, size))
        # Creating a random vector for translation part
        b = np.random.normal(0, 1, size)
        return A, b

    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    def create_transformation_matrix(size):
        np.random.seed(42)  # Ensure reproducibility
        matrix = np.eye(size) + np.random.normal(0, 0.1, (size, size))
        return matrix

    repr_shift = np.roll(repr, -1)
    repr_shift2 = np.roll(repr, -1)
    repr_shift[-1] = 0
    repr_shift2[-1] = 0
    repr_shift_add = repr + repr_shift

    repr = (repr + repr_shift_add)
    repr = (repr - repr.mean(axis=1, keepdims=True)) / repr.std(axis=1, keepdims=True)
    repr = np.dot(repr, create_transformation_matrix(repr.shape[1]))
    A, b = create_affine_transformation_matrix(repr.shape[1])
    repr = np.dot(repr, A) + b

    for row in repr:
        num_elements_to_modify = int(0.15 * len(row))
        indices_gt_0 = np.where(row > 0)[0]
        indices_le_0 = np.where(row <= 0)[0]

        swap_indices_gt_0 = np.random.choice(indices_gt_0, num_elements_to_modify, replace=False)
        swap_indices_le_0 = np.random.choice(indices_le_0, num_elements_to_modify, replace=False)
        row[swap_indices_gt_0] = 1
        row[swap_indices_le_0] = 0

    return repr.astype('float32')

In [196]:
ds_train = DefenseTransformationDataset(repr=REPR_TRAIN, labels=LABELS_TRAIN)
ds_transform = DefenseTransformationDataset(repr=transform_representations(REPR_TRAIN), labels=LABELS_TRAIN)

In [197]:
torch.use_deterministic_algorithms(False)
torch.manual_seed(0)
np.random.seed(0)

model = Task3Model(feat_size=192)
model = model.to("cuda")
dl = get_dl(ds=ds_train, batch_size=32)
finetune_classifier(model=model, dataloader=dl)

Epoch 0 loss: 0.0, accuracy: 0.8537999987602234
Epoch 1 loss: 0.0, accuracy: 0.9061999917030334
Epoch 2 loss: 0.0, accuracy: 0.9254000186920166
Epoch 3 loss: 0.0, accuracy: 0.9363999962806702
Epoch 4 loss: 0.0, accuracy: 0.9444000124931335
Epoch 5 loss: 0.0, accuracy: 0.9544000029563904
Epoch 6 loss: 0.0, accuracy: 0.9595999717712402
Epoch 7 loss: 0.0, accuracy: 0.965399980545044
Epoch 8 loss: 0.0, accuracy: 0.97079998254776
Epoch 9 loss: 0.0, accuracy: 0.972599983215332
Epoch 10 loss: 0.0, accuracy: 0.977400004863739
Epoch 11 loss: 0.0, accuracy: 0.9814000129699707
Epoch 12 loss: 0.0, accuracy: 0.9829999804496765
Epoch 13 loss: 0.0, accuracy: 0.9846000075340271
Epoch 14 loss: 0.0, accuracy: 0.9868000149726868
Epoch 15 loss: 0.0, accuracy: 0.9882000088691711
Epoch 16 loss: 0.0, accuracy: 0.9894000291824341
Epoch 17 loss: 0.0, accuracy: 0.9914000034332275
Epoch 18 loss: 0.0, accuracy: 0.9923999905586243
Epoch 19 loss: 0.0, accuracy: 0.993399977684021


In [198]:
torch.use_deterministic_algorithms(False)
torch.manual_seed(0)
np.random.seed(0)

model = Task3Model(feat_size=192)
model = model.to("cuda")
dl = get_dl(ds=ds_transform, batch_size=32)
finetune_classifier(model=model, dataloader=dl)

Epoch 0 loss: 0.0, accuracy: 0.7192000150680542
Epoch 1 loss: 0.0, accuracy: 0.819599986076355
Epoch 2 loss: 0.0, accuracy: 0.8555999994277954
Epoch 3 loss: 0.0, accuracy: 0.8794000148773193
Epoch 4 loss: 0.0, accuracy: 0.895799994468689
Epoch 5 loss: 0.0, accuracy: 0.9079999923706055
Epoch 6 loss: 0.0, accuracy: 0.9154000282287598
Epoch 7 loss: 0.0, accuracy: 0.9240000247955322
Epoch 8 loss: 0.0, accuracy: 0.9308000206947327
Epoch 9 loss: 0.0, accuracy: 0.9395999908447266
Epoch 10 loss: 0.0, accuracy: 0.944599986076355
Epoch 11 loss: 0.0, accuracy: 0.9491999745368958
Epoch 12 loss: 0.0, accuracy: 0.9538000226020813
Epoch 13 loss: 0.0, accuracy: 0.9584000110626221
Epoch 14 loss: 0.0, accuracy: 0.9621999859809875
Epoch 15 loss: 0.0, accuracy: 0.9664000272750854
Epoch 16 loss: 0.0, accuracy: 0.9688000082969666
Epoch 17 loss: 0.0, accuracy: 0.9706000089645386
Epoch 18 loss: 0.0, accuracy: 0.9751999974250793
Epoch 19 loss: 0.0, accuracy: 0.9779999852180481


In [199]:
def create_npz(repr_to_save):
    time_now_str = datetime.now().strftime("%Y%m%d_%H%M%S")
    path = f"task3_{time_now_str}.npz"
    np.savez(
        path,
        representations=repr_to_save
    )
    return path

In [200]:
path = create_npz(transform_representations(REPR_TEST))

In [201]:
def defense_submit(path_to_npz_file: str):
    SERVER_URL = "http://34.71.138.79:9090"
    TEAM_TOKEN = "8J40ASDQOjfeeSKL"

    endpoint = "/defense/submit"
    url = SERVER_URL + endpoint

    with open(path_to_npz_file, "rb") as f:
        response = requests.post(url, files={"file": f}, headers={"token": TEAM_TOKEN})
        if response.status_code == 200:
            print("Request ok")
            print(response.json())
        else:
            raise Exception(
                f"Defense submit failed. Code: {response.status_code}, content: {response.json()}"
            )

In [213]:
defense_submit(path)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))